# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [2]:
import pandas as pd 
import plotly.express as px


# Notebook Presentation

In [3]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [4]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [5]:
num_rows, num_cols = df_apps.shape
print("Number of rows:", num_rows)
print("Number of columns:", num_cols)


Number of rows: 10841
Number of columns: 12


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns. 

In [5]:
df_apps = df_apps.drop(["Last_Updated", "Android_Ver"], axis=1)


### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows. 

In [6]:
nan_ratings = df_apps[df_apps['Rating'].isna()]

df_apps_clean = df_apps.dropna(subset=['Rating'])


### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`. 


In [7]:
duplicate_instagram = df_apps_clean[df_apps_clean['App'] == 'Instagram']

df_apps_clean = df_apps_clean.drop_duplicates()


# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [8]:
highest_rated_apps = df_apps_clean.sort_values(by='Rating', ascending=False)
print(highest_rated_apps[['App', 'Rating']].head())


                      App  Rating
21    KBA-EZ Health Guide    5.00
1573       FHR 5-Tier 2.0    5.00
1096             BG Guide    5.00
1095         Morse Player    5.00
1092                DG TV    5.00


# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please? 

In [9]:
largest_apps = df_apps_clean.nlargest(5, 'Size_MBs')


# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [10]:
apps_highest_reviews = df_apps_clean.nlargest(5, 'Reviews')
print(apps_highest_reviews[['App', 'Reviews']])


                      App   Reviews
10805            Facebook  78158306
10811            Facebook  78128208
10785  WhatsApp Messenger  69119316
10797  WhatsApp Messenger  69109672
10808           Instagram  66577446


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [11]:

# Pie Chart
if 'Content_Rating' in df_apps_clean.columns:
    print("Content_Rating column exists in df_apps_clean.")
else:
    print("Content_Rating column does not exist in df_apps_clean.")
fig_pie = px.pie(df_apps_clean, names='Content_Rating', title='Content Ratings Distribution')

# Donut Chart
fig_donut = px.pie(df_apps_clean, names='Content_Rating', hole=0.5, title='Content Ratings Distribution')

# Show the charts
fig_pie.show()
fig_donut.show()


Content_Rating column exists in df_apps_clean.


# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install? 

Check the datatype of the Installs column.

Count the number of apps at each level of installations. 

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first. 

In [12]:
apps_over_1_billion = df_apps_clean[df_apps_clean['Installs'] == '1,000,000,000']
num_apps_over_1_billion = len(apps_over_1_billion)
print("Number of apps with over 1 billion installations:", num_apps_over_1_billion)


Number of apps with over 1 billion installations: 49


In [12]:
df_apps['Installs'].dtypes


dtype('O')

# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [13]:
df_apps_clean['Price'] = pd.to_numeric(df_apps_clean['Price'], errors='coerce')
most_expensive_apps = df_apps_clean.nlargest(20, 'Price')
df_apps_clean = df_apps_clean.copy()
df_apps_clean['Price'] = pd.to_numeric(df_apps_clean['Price'], errors='coerce')
df_apps_clean['Installs'] = pd.to_numeric(df_apps_clean['Installs'].astype(str).str.replace(',', ''), errors='coerce')
df_apps_clean['revenue_estimate'] = df_apps_clean['Price'] * df_apps_clean['Installs']
most_expensive_apps = df_apps_clean.nlargest(20, 'Price')








### The most expensive apps sub $250

### Highest Grossing Paid Apps (ballpark estimate)

# Plotly Bar Charts & Scatter Plots: Analysing App Categories

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [28]:
highest_competition = df_apps_clean['Category'].value_counts()

import plotly.express as px

fig = px.bar(highest_competition, x=highest_competition.index, y=highest_competition.values, title='Highest Competition by Category')
fig.show()


In [17]:
df_apps_clean['Installs'] = pd.to_numeric(df_apps_clean['Installs'].astype(str).str.replace(',', ''), errors='coerce')
median_installs_free = df_apps_clean[df_apps_clean['Type'] == 'Free']['Installs'].median()
median_installs_paid = df_apps_clean[df_apps_clean['Type'] == 'Paid']['Installs'].median()

median_installs_free, median_installs_paid


(500000.0, 5000.0)

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [27]:
import plotly.express as px

fig = px.bar(highest_competition, x=highest_competition.values, y=highest_competition.index, orientation='h', title='Highest Competition by Category')
fig.show()


### Category Concentration - Downloads vs. Competition

**Challenge**: 
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this. 

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log') 

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). 


In [19]:
num_types = df_apps_clean['Type'].nunique()
print("Number of different types:", num_types)
df_apps_clean['Genres'].value_counts()



Number of different types: 2


Genres
Tools                         732
Entertainment                 498
Education                     446
Action                        349
Productivity                  334
                             ... 
Health & Fitness;Education      1
Puzzle;Education                1
Card;Brain Games                1
Lifestyle;Education             1
Lifestyle;Pretend Play          1
Name: count, Length: 115, dtype: int64

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data? 

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/). 

* Find a way to set the colour scale using the color_continuous_scale parameter. 
* Find a way to make the color axis disappear by using coloraxis_showscale. 

# Grouped Bar Charts: Free vs. Paid Apps per Category

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart: 

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category. 

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value). 

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart. 

<img src=https://imgur.com/uVsECT3.png>


# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below: 

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories. 

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [26]:
import warnings
 
 
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    median_price = df_apps_clean[df_apps_clean['Type'] == 'Paid']['Price'].median()

